In [1]:
from sleeper_wrapper import League
from sleeper_wrapper import Stats
import statistics
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

LEAGUE_ID = "649912836461539328"
CURRENT_WEEK = 9
FORMAT = "half_ppr"



# Fantasy Analysis for Week 7

Hello boys, in lieu of power rankings this week I will be doing a bit of analysis and taking a look behind the scenes at weekly point scoring and scoring trends throughout the season. As a reminder, league standings are currently as follows:

```
Noe's Loyal Lighting Salesman Division
1. Nahome (11-3)
2. Noe (8-6)
3. Will (7-7)
4. Ben (6-8)
5. Stephane (1-13)

Noah's SideThiccs
1. Noah (9-5)
2. Praveen (9-5)
3. Stian (8-6)
4. Tom (6-8)
5. Tegran (5-9)
```

This weeks power rankings that I released on Tuesday had pretty little movement, with Nahome still at the top and Stephane still in the cellar. He is hoping to get out of the last spot with a blockbuster trade this week, sending CMC and the Gasman to Ben in exchange for Mr. Hollywood Samuel and Sleepy Joe Mixon. 

```
1. Nahome
2. Praveen
3. Noah
4. Noe
5. Stian
6. Will
7. Tom
8. Ben
9. Tegran
10. Stephane
```

If the playoffs started today, we would have ```Praven vs. Will``` and ```Stian vs. Noe``` in the opening rounds, with Noah and Nahome both earning byes. In the toilet bowl, we would have ```Tegran vs. Ben``` and ```Stephane vs. Tom```.

This document will focus on going behind the summary stats on the standings page and look into team consistency, luck, and the level of the opposition. Hopefully it will be interesting and a good way to get me back into doing weekly writeups!

In [2]:
def pair_user_rosters(rosters):
    """pairs a users roster id with their user id"""
    pairings = {}
    
    for roster in rosters:
        pairings[roster["roster_id"]] = roster["owner_id"]

    return pairings

def pair_usernames(users):
    pairings = {}

    # this is stored dumb, but basically team name can be in metadata or it can
    # be outside of it. If a team has no name, we can just use their display name
    for user in users:
        try:
            pairings[user["user_id"]] = user["metadata"]["team_name"]
        except KeyError: # stored dumb
            try:
                pairings[user["user_id"]] = user["team_name"]
            except KeyError: # no team name
                pairings[user["user_id"]] = user["display_name"]

    return pairings

    

def pair_matchups(games):
    """pair matchup data with users and their opponent"""
    scoreboards = {}

    for player in games:
        if player["matchup_id"] in scoreboards.keys():
            scoreboards[player["matchup_id"]][ROSTER_PAIRINGS[player["roster_id"]]] = player
        else:
            scoreboards[player["matchup_id"]] = {ROSTER_PAIRINGS[player["roster_id"]]: player}

    return scoreboards


In [3]:
league = League(LEAGUE_ID)

users = league.get_users()
rosters = league.get_rosters()

ROSTER_PAIRINGS = pair_user_rosters(rosters)
NAME_PAIRINGS = pair_usernames(users)

In [4]:
all_games = {}

# creates a dictionary with all matchups in the league through CURRENT_WEEK
for week in range(1, CURRENT_WEEK+1):
    all_games[week] = pair_matchups(league.get_matchups(week))

In [5]:
# sanity check the weeks and uncomment here if you wanna
# for week in all_games:
#     print(f"------------Week {week}------------")
#     for matchup in all_games[week]:
#         for team in all_games[week][matchup]:
#             print(f" {NAME_PAIRINGS[str(team)]}: {all_games[week][matchup][team]['points']}")
#         print()

In [6]:
# how does median points scored differ from points per game?
# lets define breakout as 30+ points
BREAKOUT_CUT = 30
#   league discussion has been on points for (as an aside, how about opp. median score?)
# ADD STATS STUFF TO THIS FUNCTION BUT KEEP THIS UP TO DATE
STATS_FIELDS = ["Weeks", "Weekly Points For", "Weekly Points Against", "Points For", "Points Against", "Points per Game", 
                "Median Points Scored", "Points Against per Game", "Median Points Against", "3 game rolling average", 
                "5 game rolling average", "Breakout Games", "Breakout Games Against", "Game Margin", "Blowout Index"]

# blowout index: closer to 0 is close games, bigger numbers indicate larger losses and wins

#   TODO: edge case for less than 3 or 5 weeks?

def team_stats(games):
    """calculate team stats. returns a dictionary of team ids with associated
       lists that define team stats. list has order:
       [total weeks, weekly points for, weekly points against, total points for, total points against, ppg, 
       median points scored, papg, median points against, 3 game rolling avg. points scored, 
       5 game rolling avg. points scored, breakouts for, breakouts against, game margin, blowout index]
    """
    weeks = len(games.keys())
    final_stats = {}
    weekly_pf = {}
    weekly_pa = {}
    weekly_bf = {}
    weekly_ba = {}

    for user in NAME_PAIRINGS:
        final_stats[user] = []
        weekly_pf[user] = []
        weekly_pa[user] = []
        weekly_bf[user] = []
        weekly_ba[user] = []

    for week in games:
        for matchup in games[week]:
            teams = list(games[week][matchup].keys())
            team0 = games[week][matchup][teams[0]]
            team1 = games[week][matchup][teams[1]]
            # use team data to fill in pf and opponent data for pa
            weekly_pf[teams[0]].append(team0["points"])
            weekly_pf[teams[1]].append(team1["points"])
            weekly_pa[teams[0]].append(team1["points"])
            weekly_pa[teams[1]].append(team0["points"])
            weekly_bf[teams[0]].append({})
            weekly_bf[teams[1]].append({})
            weekly_ba[teams[0]].append({})
            weekly_ba[teams[1]].append({})
            for player0, player1 in zip(team0["players_points"].keys(), team1["players_points"].keys()):
                if team0["players_points"][player0] >= BREAKOUT_CUT:
                    weekly_bf[teams[0]][week-1][player0] = team0["players_points"][player0]
                    weekly_ba[teams[1]][week-1][player0] = team0["players_points"][player0]
                elif team1["players_points"][player1] >= BREAKOUT_CUT:
                    weekly_ba[teams[0]][week-1][player1] = team1["players_points"][player1]
                    weekly_bf[teams[1]][week-1][player1] = team1["players_points"][player1]

    for user in NAME_PAIRINGS:
        final_stats[user].append(weeks)
        final_stats[user].append(weekly_pf[user])
        final_stats[user].append(weekly_pa[user])
        pf = sum(weekly_pf[user])
        pa = sum(weekly_pa[user])
        final_stats[user].append(round(pf, 2))
        final_stats[user].append(round(pa, 2))
        final_stats[user].append(round(pf/weeks, 2))
        final_stats[user].append(round(statistics.median(weekly_pf[user]), 2))
        final_stats[user].append(round(pa/weeks, 2))
        final_stats[user].append(round(statistics.median(weekly_pa[user]), 2))


        rolling_3 = []
        for i in range(weeks):
            if i < 3:
                rolling_3.append(round(sum(weekly_pf[user][0:i+1])/(i+1), 2))
            else:
                rolling_3.append(round(sum(weekly_pf[user][i-2:i+1])/3, 2))
        final_stats[user].append(rolling_3)

        rolling_5 = []
        for i in range(weeks):
            if i < 5:
                rolling_5.append(round(sum(weekly_pf[user][0:i+1])/(i+1), 2))
            else:
                rolling_5.append(round(sum(weekly_pf[user][i-4:i+1])/5, 2))
        final_stats[user].append(rolling_5)

        final_stats[user].append([len(k.keys()) for k in weekly_bf[user]])
        final_stats[user].append([len(k.keys()) for k in weekly_ba[user]])
        final_stats[user].append([round(weekly_pf[user][week] - weekly_pa[user][week], 2) for week in range(weeks)])
        final_stats[user].append(round(sum([final_stats[user][-1][i]**2 for i in range(weeks)]) / weeks, 2)) # mean squared margin
        
        # ADD HERE

    return final_stats

stats = team_stats(all_games)

## Scoring Heatmap
Now that we have grabbed the weekly results and generate team stats, lets have a look at everyone's weekly scoring. Darker blues are higher scores, whereas darker reds are lower scores. The first heatmap shows these scores, whereas the second shows weekly scoring ranks. In this second heatmap, darker browns are better ranks whereas darker greens are worse ranks. The transition between green and brown marks the transition from being above/below the league median score for the week.

In [7]:
weekly_scores = [stats[team][1] for team in stats.keys()]

red_mid_blue_scale = [[0.0, "rgb(165,0,38)"],
             [0.1111111111111111, "rgb(215,48,39)"],
             [0.2222222222222222, "rgb(244,109,67)"],
             [0.3333333333333333, "rgb(253,174,97)"],
             [0.4444444444444444, "rgb(254,224,144)"],
             [0.5555555555555556, "rgb(224,243,248)"],
             [0.6666666666666666, "rgb(171,217,233)"],
             [0.7777777777777778, "rgb(116,173,209)"],
             [0.8888888888888888, "rgb(69,117,180)"],
             [1.0, "rgb(49,54,149)"]]

fig  = go.Figure(data = go.Heatmap(
                            z = weekly_scores,
                            y = list(NAME_PAIRINGS.values()),
                            x = [f"Week {week}" for week in range(1, CURRENT_WEEK+1)],
                            colorscale= red_mid_blue_scale,
                            colorbar = dict(title= "Points Scored"),
                            )
                )

fig.update_layout(
    title={
        'text': "Points Scored By Week",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.show()

In [8]:
weekly_rank = [[] for i in range(len(NAME_PAIRINGS.keys()))]

for idx in range(CURRENT_WEEK):
    scores = [team_score[idx] for team_score in weekly_scores]
    ranks = [sorted(scores, reverse=True).index(score)+1 for score in scores]
    for i in range(len(weekly_rank)):
        weekly_rank[i].append(ranks[i])
        
fig  = go.Figure(data = go.Heatmap(
                            z = weekly_rank,
                            y = list(NAME_PAIRINGS.values()),
                            x = [f"Week {week}" for week in range(1, CURRENT_WEEK+1)],
                            colorscale= px.colors.diverging.BrBG,
                            colorbar = dict(title= "Rank"),
                            )
                )

fig.update_layout(
    title={
        'text': "Scoring Rankings by Week",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.show()

## Point Scoring Rankings

The key discussion that spawned this analysis was with regards to the points for statistic. Because it can be skewed by high scoring weeks that only count for one win. The coming series of analyses drill down into additional point statistics and take a look at how these differ from the monolithic points for stat. First I am going to compare the points per game stat (mean points scored in a week) with the median points scored in a week. This should better insulate the statistic from high point fluctuations and give a more stable indicator of performance on a week to week basis.

Generally speaking, we see higher mean scores than median scores, which makes sense considering that our scores tend to skew off to the right (many scores are between 90-120, less are between 140-180). Stian is a special case, with a median score higher than his average score. Looking again at the heatmap above, notice he has been either the highest or second highest scorer or he has been the lowest scorer in the week. These low scores actually hold his average score down as compared to someone like Tegran, who has scored super high once and been under the median most of the time. 

Stian's boom or bust cycle also led me towards another statistic, which I will refer to as the blowout index. This is inspired by the Mean Squeared Error statistic from regression analyses. I am interested in the degree to which a team has close games vs. blowout games, since it seems like basically all of Stian's games are him winning by a ton or losing by a ton. To calculate this stat, I square the margin of victory/defeat (this increases the effect size of huge victories or losses), and then sum the results. This statistic is then divided by the number of weeks and results in the mean squared game margin for a specific team. After the median and mean score analyses, you can find a bar chart for each team showing the weekly margin of victory and then an aggregated chart showing this mean squared game margin (Blowout Index) statistic. Hint hint - Stian is basically off the charts.

In [9]:
# weekly scores

interested_in = STATS_FIELDS.index("Median Points Scored")
weekly_medians = {}
for team in stats:
    weekly_medians[stats[team][interested_in]] = NAME_PAIRINGS[team]

print("Medians")
for num, score in enumerate(sorted(weekly_medians.keys(), reverse=True)):
    print(f"{num+1}. {weekly_medians[score]}: {score} pts")

interested_in = STATS_FIELDS.index("Points per Game")
weekly_means = {}
for team in stats:
    weekly_means[stats[team][interested_in]] = NAME_PAIRINGS[team]

print("\n Means")
for num, score in enumerate(sorted(weekly_means.keys(), reverse=True)):
    print(f"{num+1}. {weekly_means[score]}: {score} pts")
    
interested_in = STATS_FIELDS.index("Blowout Index")
blowout_idx = {}
for team in stats:
    blowout_idx[stats[team][interested_in]] = NAME_PAIRINGS[team]

print("\n Mean Squared Victory Margin")
for num, score in enumerate(sorted(blowout_idx.keys(), reverse=True)):
    print(f"{num+1}. {blowout_idx[score]}: {score}")

Medians
1. tealeaves: 157.08 pts
2. AssNTitties: 140.84 pts
3. Tree Leaves: 131.82 pts
4. Thicc King: 128.18 pts
5. My Goat Loves You: 127.6 pts
6. billcap: 126.5 pts
7. the noés: 120.32 pts
8. SideThicc #2: 118.44 pts
9. The Brutherhood: 116.1 pts
10. RB sanctuary: 110.04 pts

 Means
1. AssNTitties: 142.19 pts
2. Thicc King: 133.19 pts
3. tealeaves: 131.47 pts
4. My Goat Loves You: 129.81 pts
5. Tree Leaves: 129.03 pts
6. billcap: 126.55 pts
7. The Brutherhood: 126.23 pts
8. SideThicc #2: 122.19 pts
9. the noés: 121.3 pts
10. RB sanctuary: 104.61 pts

 Mean Squared Victory Margin
1. tealeaves: 3442.5
2. RB sanctuary: 1884.35
3. Tree Leaves: 1545.21
4. My Goat Loves You: 1338.06
5. billcap: 1008.85
6. SideThicc #2: 709.58
7. AssNTitties: 591.15
8. The Brutherhood: 549.55
9. Thicc King: 325.6
10. the noés: 186.96


In [10]:
fig = go.Figure()

fig.add_trace(
    go.Bar(x=list(weekly_means.values()), y=list(weekly_means.keys()), 
           marker=dict(color=list(weekly_means.keys()), coloraxis="coloraxis"))
)

fig.update_layout(coloraxis=dict(colorscale=px.colors.diverging.RdYlGn), 
                  title="Mean Score per Week", 
                  showlegend=False, 
                  xaxis={'categoryorder':'total descending'},
                 )

fig.show()

In [11]:
fig = go.Figure()

fig.add_trace(
    go.Bar(x=list(weekly_medians.values()), y=list(weekly_medians.keys()), 
           marker=dict(color=list(weekly_medians.keys()), coloraxis="coloraxis"))
)

fig.update_layout(coloraxis=dict(colorscale=px.colors.diverging.RdYlGn), 
                  title="Median Score per Week", 
                  showlegend=False, 
                  xaxis={'categoryorder':'total descending'},
                 )

fig.show()

In [12]:
fig = make_subplots(rows=5, cols=2, shared_yaxes=True, subplot_titles=list(NAME_PAIRINGS.values()))

for i in range(5):
    for j in range(1, 3):
        fig.add_trace(
            go.Bar(y=stats[list(stats.keys())[(i+i+j)-1]][STATS_FIELDS.index("Game Margin")], 
                   x=[f"Week {week}" for week in range(1, CURRENT_WEEK+1)], 
                   marker=dict(color=list(stats[list(stats.keys())[(i+i+j)-1]][STATS_FIELDS.index("Game Margin")]), 
                               coloraxis="coloraxis")),
            i+1, j
        )

fig.update_layout(coloraxis=dict(colorscale=px.colors.diverging.RdYlGn), 
                  title="Game Margins",
                  showlegend=False, 
                  height=1000, width=950
                 )

fig.show()

In [13]:
fig = go.Figure()

fig.add_trace(
    go.Bar(x=list(blowout_idx.values()), y=list(blowout_idx.keys()), 
           marker=dict(color=list(blowout_idx.keys()), coloraxis="coloraxis"))
)

fig.update_layout(coloraxis=dict(colorscale=px.colors.sequential.Viridis), 
                  title="Mean Squared Victory Margin", 
                  showlegend=False, 
                  xaxis={'categoryorder':'total descending'},
                 )

## Smoothing Point Trends

The final thing I wanted to consider was smoothing out point trends with a moving average. I'll consider both a three week and a five week moving average, but its a bit tough for fantasy since there are so few data points on which to consider. Below you will see scatterplots showing the points scored in a given week, and then the moving average of points scored per week overlayed on top. Positively sloped lines indicate a team is trending towards scoring more points, whereas a negative slope indicates the opposite. Hover over a point for details, the orange bar represents the 3 game average while the green bar is the 5 game average.

In [14]:
fig = make_subplots(rows=5, cols=2, shared_yaxes=True, subplot_titles=list(NAME_PAIRINGS.values()))

for i in range(5):
    for j in range(1, 3):           
        fig.add_trace(
            go.Scatter(x=[f"Week {week}" for week in range(1, CURRENT_WEEK+1)], 
                       y=stats[list(stats.keys())[(i+i+j)-1]][STATS_FIELDS.index("3 game rolling average")], 
                       line=dict(color='orange', width=1),
                       name="3 Game Avg."),
            i+1, j
        )
        
        fig.add_trace(
            go.Scatter(x=[f"Week {week}" for week in range(1, CURRENT_WEEK+1)], 
                       y=stats[list(stats.keys())[(i+i+j)-1]][STATS_FIELDS.index("5 game rolling average")], 
                       line=dict(color='green', width=1),
                       name="5 Game Avg."),
            i+1, j
        )
        
        fig.add_trace(
            go.Scatter(x=[f"Week {week}" for week in range(1, CURRENT_WEEK+1)], 
                       y=stats[list(stats.keys())[(i+i+j)-1]][STATS_FIELDS.index("Weekly Points For")],
                       mode="markers",
                       marker_color='rgba(152, 0, 0, .8)',
                       name="Score"),
            i+1, j
        )

fig.update_layout(title="Points Scored with Moving Averages",
                  height=1000, 
                  showlegend=False,
                  width=950)

fig.show()

## Raw Data

If you would like to see more raw printouts of the data I used and generated, run the cell below.

In [15]:
# lets print these out nicely to sanity check.
for team in stats:
    print(f"\n----------- {NAME_PAIRINGS[team]} -----------")
    for name, stat in zip(STATS_FIELDS, stats[team]):
        if name == "3 game rolling average":
            print("3 Game Rolling Average Points Scored")
            for i in range(len(stat)):
                print(f"\tWeeks {i+1}-{i+4}: {stat[i]} ppg")
        elif name == "5 game rolling average":
            print("5 Game Rolling Average Points Scored")
            for i in range(len(stat)):
                print(f"\tWeeks {i+1}-{i+6}: {stat[i]} ppg")
        elif name == "Breakout Games":
            print(f"Total Breakout Games ({BREAKOUT_CUT} pts): {sum(stat)}")
            for i in range(len(stat)):
                print(f"\tWeek {i}: {stat[i]} breakouts")
        elif name == "Breakout Games Against":
            print(f"Total Breakout Games Against ({BREAKOUT_CUT} pts): {sum(stat)}")
            for i in range(len(stat)):
                print(f"\tWeek {i}: {stat[i]} breakouts against")
        elif name == "Game Margin":
            print(f"Total Margin: {sum(stat)} pts")
            for i in range(len(stat)):
                print(f"\tWeek {i}: {stat[i]} pts")
        else:
            print(f"{name}: {stat}")



----------- My Goat Loves You -----------
Weeks: 7
Weekly Points For: [171.86, 127.6, 108.52, 109.8, 113.5, 141.54, 135.86]
Weekly Points Against: [113.42, 108.56, 152.14, 159.12, 114.26, 163.22, 107.88]
Points For: 908.68
Points Against: 918.6
Points per Game: 129.81
Median Points Scored: 127.6
Points Against per Game: 131.23
Median Points Against: 114.26
3 Game Rolling Average Points Scored
	Weeks 1-4: 171.86 ppg
	Weeks 2-5: 149.73 ppg
	Weeks 3-6: 135.99 ppg
	Weeks 4-7: 115.31 ppg
	Weeks 5-8: 110.61 ppg
	Weeks 6-9: 121.61 ppg
	Weeks 7-10: 130.3 ppg
5 Game Rolling Average Points Scored
	Weeks 1-6: 171.86 ppg
	Weeks 2-7: 149.73 ppg
	Weeks 3-8: 135.99 ppg
	Weeks 4-9: 129.44 ppg
	Weeks 5-10: 126.26 ppg
	Weeks 6-11: 120.19 ppg
	Weeks 7-12: 121.84 ppg
Total Breakout Games (30 pts): 4
	Week 0: 1 breakouts
	Week 1: 1 breakouts
	Week 2: 0 breakouts
	Week 3: 1 breakouts
	Week 4: 0 breakouts
	Week 5: 1 breakouts
	Week 6: 0 breakouts
Total Breakout Games Against (30 pts): 4
	Week 0: 0 breakouts

In [16]:
# Aiyuk is ASS